<a href="https://colab.research.google.com/github/esfernandezb/Examen/blob/main/U2_4_Comunicacion_remota_RPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RPC

La facilidad de RPC usa un esquema de pasaje de mensajes para intercambiar información entre los procesos llamados (proceso cliente) y llamado (proceso servidor). Normalmente el proceso servidor duerme, esperando la llegada de un mensaje de requerimiento. El proceso cliente se bloquea cuando envía el mensaje de requerimiento hasta recibir la respuesta.
Existen diversas formas de implementar RPC. Vamos a revisar las mas usadas:

Referencias:

https://stackoverflow.com/questions/1879971/what-is-the-current-choice-for-doing-rpc-in-python

RPC simple

https://docs.python.org/es/3/library/xmlrpc.client.html

https://rico-schmidt.name/pymotw-3/xmlrpc.server/index.html

https://tldp.org/HOWTO/XML-RPC-HOWTO/xmlrpc-howto-competition.html

RPyC

https://rpyc.readthedocs.io/en/latest/tutorial/tut1.html

https://code-maven.com/rpc-with-python-using-rpyc



## RPC simple

XML-RPC es la forma mas sencilla y clasica de realizar comunicacion RPC. Actualmente es muy insegura, por lo que se recomienda realizar pruebas en un entorno con elementos agregados de seguridad.
Pirmero empezamos configurando variables iniciales.

In [ ]:
run_thread = True
first_time = True
server = None

Definidos nuestro servidor RPC de la siguiente manera:
*  La funcion list_contents devuelve una lista de directorios que se encuentran dentro del servidor donde se esta realizando la peticion
* La funcion run_server es iniciar el servidor. El modulo SimpleXMLRPCServer me permite realizar esta accion con dos parametros: el destino y el puerto de conexion. El parametro logRequest permite recibir las peticiones tipo log por parte del cliente. La funcion register_function debe estar registrada en nuestro objeto serverRPC para que pueda ejecutarse. Luego de ello viene la ejecucion.
* Las funciones stop_server y start_server funcionan de manera similar como en el ejemplo de comunicacion directa con sockets.

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
import logging
import os
from multiprocessing import Process

def list_contents(dir_name):
    logging.info('list_contents(%s)', dir_name)
    return os.listdir(dir_name)

def run_server():
  logging.basicConfig(level=logging.INFO)

  serverRPC = SimpleXMLRPCServer(
      ('localhost', 9000),
      logRequests=True,
  )

  serverRPC.register_function(list_contents)
  print('The server is ready to receive')
  while 1:
      try:
          print('Use Control-C to exit')
          serverRPC.serve_forever()
      except KeyboardInterrupt:
          print('Exiting')

def stop_server():
  global server
  if server is not None:
    server.terminate()
    server.join()

def start_server(run_thread):
  global server
  if run_thread:
    server = Process(target=run_server)
    server.start()
  else:
    run_server()

Iniciamos nuestro servidor.

In [ ]:
stop_server()
start_server(run_thread)
run_thread = False

The server is ready to receive
Use Control-C to exit


Ejecutamos el cliente. Observe el resultado. Que conclusiones podemos obtener? que pasaria si ejecutamos este codigo en otro entorno?

In [ ]:
import xmlrpc.client

proxy = xmlrpc.client.ServerProxy('http://localhost:9000')
print(proxy.list_contents('/tmp'))

['python-languageserver-cancellation', 'initgoogle_syslog_dir.0', 'dap_multiplexer.367014a92ca6.root.log.INFO.20220620-235642.42', 'debugger_1n5wbwezny', 'dap_multiplexer.INFO', 'pyright-95-esvyNNrPfLrM', 'pyright-95-gEXbDKrh0uvl']


##RPyC



RPyC provee metodos de comunicacion remota basados en Python. 

En ambas máquinas necesitamos tener instalado Python y el paquete RPyC. En el servidor necesitamos lanzar un proceso antes de poder comunicarnos con él. La ventaja es que funciona en los 3 principales sistemas operativos: Linux, Mac OSX, y MS Windows, y se puede hacer RPyC entre máquinas que corren diferentes OS.

Primero procedemos con la instalacion de la libreria.

In [ ]:
!pip install rpyc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 117 kB 62.6 MB/s 


### Implementacion clasica

Existe una implementacion clasica y otra moderna del codigo de la libreria. Revisemos la clasica primero. Podemos iniciar un servidor de forma sencilla con la siguiente linea de codigo.

In [ ]:
!python rpyc_classic.py localhost

El problema con el entorno de Colab es que no es sencillo acceder al entorno virtual de Python. En este caso trabajaremos en una maquina local. Solo basta ubicar donde esta el script y ejecutarlo con el comando escrito arriba.

El resultado se vera de la siguiente manera:

INFO:SLAVE/18812:server started on [127.0.0.1]:18812

En donde:
This shows the parameters this server is running with:

* SLAVE indica el SlaveService
* [127.0.0.1]:18812 es la direccion de donde se aceptan conexiones, en este caso, solo seran del localhost. Si se inicia un servidor con --host 0.0.0.0, se podran ejecutar peticiones de cualquier lado.

Ya con esto, podemos empezar a ejecutar ordenes en nuestro servidor. El siguiente script es un ejemplo sencillo de interaccion.

In [ ]:
# rpyc_modules.py

import rpyc
import sys

# Si no envio parametros, se mostrara el siguiente mensaje con el nombre del script
if len(sys.argv) < 2:
   exit("Usage {} SERVER".format(sys.argv[0]))
 
server = sys.argv[1]

# Para ejecutarlo debo enviar la direccion de conexion
# Ejemplo: python rpyc_modules.py localhost
# Primero realizamos la conexion al servidor remoto
conn = rpyc.classic.connect(server)

# Con la conexion realizada, procedemos a ejecutar comandos en el servidor
# Podemos adjuntar un nombre local a un objeto Python remoto. Por ejemplo, esta línea conecta el nombre rsys en nuestro cliente con el objeto sys en el servidor. (Importa automáticamente el módulo sys en el servidor).
rsys = conn.modules.sys

# Una vez que hemos hecho esa llamada podemos usar rsys en el cliente igual como usaríamos el sys en el servidor. Por ejemplo podemos acceder a sus atributos e imprimirlos en la consola del cliente:
print(rsys.version)

# Podemos hacer lo mismo con el modulo os
ros = conn.modules.os
print('Name: ', ros.name)

El siguiente codigo me permite manejar variables entre un cliente y un servidor

In [ ]:
# rpyc_variables.py

import rpyc
import sys
 
if len(sys.argv) < 2:
   exit("Usage {} SERVER".format(sys.argv[0]))
 
server = sys.argv[1]
 
conn = rpyc.classic.connect(server)
# execute ejecutara un codigo en el servidor
# aqui solo creamos una variable y le cambiamos su valor
conn.execute('x = 21')
conn.execute('x *= 2')
print(conn.eval('x'))     # 42

# podemos hacer lo mismo con estructuras de datos
conn.execute('scores = { "Foo" : 10 }')
conn.execute('scores["Foo"] += 1')
conn.execute('scores["Bar"] = 42')
 
# eval me permite copiar un elemento al cliente
local_scores = conn.eval('scores')
print(local_scores)         # {'Foo': 11, 'Bar': 42}
print(local_scores['Foo'])  # 11
 
# namespace me permite usar elementos en la maquina remota. Se esta tomando la llave del diccionario y modificando uno de sus valores.
conn.namespace["scores"]["Bar"] += 58
print(conn.eval('scores'))  # {'Foo': 11, 'Bar': 100}

print(local_scores)


El siguiente codigo se maneja como un script completo. Vamos a ejecutarlo en el servidor y recibir una respuesta desde el cliente. Iniciamos con el script a ejecutar, el cual presenta el resultado de una serie de Fibonacci.

In [ ]:
# rpyc_remote_code.py
# Calculo sencillo de una serie de Fibonacci

def fib(n):
    if n == 1:
        return [1]
    if n == 2:
        return [1, 1]
 
    values = [1, 1]
    for _ in range(2, n):
        values.append(values[-1] + values[-2])
    return values

La funcion devuelve un arreglo con los resultados de la serie.

In [ ]:
fib(8)

[1, 1, 2, 3, 5, 8, 13, 21]

Esta funcion se debe guardar en un script y almacenarlo en el servidor. EL codigo que ejecuta el script debe encontrarse en la misma ruta. Este codigo es el siguiente:

In [ ]:
import rpyc
import sys

if len(sys.argv) < 2:
   exit("Usage {} SERVER".format(sys.argv[0]))

server = sys.argv[1]

filename = 'rpyc_remote_code.py'

conn = rpyc.classic.connect(server)
with open(filename) as fh:
    my_code = fh.read()
    conn.execute(my_code)

rfib = conn.namespace['fib']
print(rfib(1))  # [1]
print(rfib(2))  # [1, 1]
print(rfib(5))  # [1, 1, 2, 3, 5]

### Implementacion moderna

La implementacion moderna esta basada en servicios. En el modo clásico, el cliente obtiene básicamente el control total sobre el servidor. Los servicios proporcionan una manera de exponer un conjunto bien definido de capacidades a la otra parte, lo que hace de RPyC una plataforma RPC genérica.

Los servicios son muy sencillos. Observe la siguiente clase, que nota en ella?

In [ ]:
# rpyc_base.py
import rpyc

class MyService(rpyc.Service):
    def on_connect(self, conn):
        # code that runs when a connection is created
        # (to init the service, if needed)
        pass

    def on_disconnect(self, conn):
        # code that runs after the connection has already closed
        # (to finalize the service, if needed)
        pass

    def exposed_get_answer(self): # this is an exposed method
        return 42

    exposed_the_real_answer_though = 43     # an exposed attribute

    def get_question(self):  # while this method is not exposed
        return "¿cuál es la velocidad de una golondrina sin que lleve peso?"

from rpyc.utils.server import ThreadedServer
t = ThreadedServer(MyService, port=18861)
t.start()

Como pueden ver, aparte de los métodos especiales de inicialización/finalización, hay libertad de definir la clase como cualquier otra clase. Sin embargo, a diferencia de las clases normales, pueden elegir qué atributos serán expuestos a la otra parte: si el nombre comienza con exposed_, el atributo será accesible remotamente, de lo contrario, sólo será accesible localmente.

El siguiente codigo realiza consumos al servicio. 

In [ ]:
# rpyc_consulta_base.py
import rpyc
import sys

if len(sys.argv) < 2:
   exit("Usage {} SERVER".format(sys.argv[0]))
 
server = sys.argv[1]

# La conexion se realiza por el servicio creado en el script
conn = rpyc.connect(server, 18861)

# El servicio es expuesto como la raiz del objeto conexion
print(conn.root)

# Este objeto raiz es una referencia a la instancia del servicio que esta ejecutandose en el servidor. Esta puede ser accesada  por el usuario e invocar sus atributos y metodos.
print(conn.root.get_answer())
print(conn.root.the_real_answer_though)

# Este metodo no puede ser accedido
print(conn.root.get_question())

Con esto ya estamos consumiento el servicio con el objeto. Como hariamos para ejecutar un script en otro archivo?